<a id='cardiac'></a>

# Cardiac Developmental Atlas Option

The Cardiac Developmental Atlas Option allows the user to predict the cell types in their own cardiac dataset based on the cardiac developmental atlas used in our paper (Galdos, Xu, et al. 2021). 

It can be accessed through the **`cardiacDevAtlas`** mode available in **predictOne** and **predictAll** `layerObjectPaths` argument. 

## REQUIRED USER INPUTS:

- **runMode**: predictOne or predictAll
- **predNormExpr**: normalized expression matrix csv file
- **rejectionCutoff**: float between 0 and 1 denoting the minimum probability for a prediction to not be rejected
- **timePoint**: float between 7.5 and 14
    - Timepoints present within the cardiac developmental atlas include the following:
        - E7.75
        - E8.25
        - E9.25
        - E10.5
        - E13.5
    - Any timepoint between E7.5 and E14 is rounded to the nearest timepoint in the selection above.

##### EXAMPLE: 
`cellpy --runMode predictAll --rejectionCutoff 0.5 --predNormExpr /cellpy_example/cardiac_normalized.csv --layerObjectPaths cardiacDevAtlas --timePoint 13`


## Analysis of cardiacDevAtlas option in R

The following example uses the **predictAll** CellPy option with **`cardiacDevAtlas`** in the `layerObjectPaths` argument. The query dataset in this example is the dataset (Galdos, Xu, et al. 2021) using mouse E10.5 hearts that were collected in this manuscript. 

In [ ]:
#R-analysis
library(Seurat)
library(dplyr)
numcols = max(count.fields("~/Downloads/predictionall_reject0.5.csv", sep = ','))

prediction = read.table("~/Downloads/predictionall_reject0.5.csv", header = FALSE, sep = ",", 
                        col.names = paste0("V",seq_len(numcols)), fill = TRUE)

cellpy_preds = data.frame(prediction)
head(cellpy_preds)

#replace blanks in dataframe with <NA>
cellpy_preds1 <- cellpy_preds
cellpy_preds1[cellpy_preds1 == ""] <- NA 
head(cellpy_preds1)

galdos = readRDS("~/Downloads/galdos_final.object_05172021.rds")

galdos_barcodes = rownames(galdos@meta.data)

cellpy_preds2 <- cellpy_preds1[ cellpy_preds1$V1 %in% galdos_barcodes, ]

#match order of barcodes with seurat object
cellpy_preds3 = cellpy_preds2[match(galdos_barcodes, cellpy_preds2$V1), ] 


galdos[["Celltype1_pred_50"]] <- cellpy_preds3$V2
galdos[["Celltype2_pred_50"]] <- cellpy_preds3$V3
galdos[["Celltype3_pred_50"]] <- cellpy_preds3$V4

In [ ]:
DimPlot(galdos, group.by="Celltype1_pred_50", reduction = "umap", pt.size = 0.5) 

In [ ]:
[TODO: add figures for 0.5 and 0.8 side by side!!!!!!!!!!!!!!!!!!!!!!!!!]

In [ ]:
DimPlot(galdos, group.by="Celltype3_pred_50", reduction = "umap", pt.size = 0.5) 

In [ ]:
[TODO: add figures for 0.5 and 0.8 side by side!!!!!!!!!!!!!!!!!!!!!!!!!]

In [ ]:
DimPlot(galdos, group.by="Celltype3_pred_50", reduction = "umap", pt.size = 0.5) 

In [ ]:
[TODO: add figures for 0.5 and 0.8 side by side!!!!!!!!!!!!!!!!!!!!!!!!!]

In [ ]:
#heatmaps
Idents(galdos) <- galdos$Celltype1_pred_50

galdos.markers <- FindAllMarkers(galdos, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
galdos.markers %>% group_by(cluster) %>% top_n(n = 2, wt = avg_log2FC)
top10 <- galdos.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_log2FC)

png(filename="~/Desktop/galdos_Celltype1_pred_50_heatmap.png", width=1400, height=1100)
par(mar=c(5,6,4,100)+.1)
DoHeatmap(galdos, features = top10$gene, size = 6, angle=90) + NoLegend()
dev.off()

In [ ]:
[TODO: add heatmaps!!!!!!!!!!!!]

Comparison with heatmap generated when datasets ran with a predictAll rejection cutoff of 0.8:

In [ ]:
[TODO: add heatmaps!!!!!!!!!!!!]